In [24]:
import os
import glob
import sys
import copy
import zlib
import pathlib
import tempfile
import subprocess
import itertools
import random
import json
import math
import scipy as sp
import scipy.optimize
import cma
import skimage.draw
from scipy.optimize import differential_evolution, minimize, basinhopping
from sklearn.metrics import pairwise_distances
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from skimage.segmentation import flood_fill

sys.path.append("../src")

from anno_designer import ad_to_grid, BLOCKING, object_positions, ID_TOWNHALL, ID_GUILDHOUSE
from utils import crop_to_nonzero, TOWNHALL_COVERAGE

In [25]:
input_glob = os.path.join("..", "resources", "Anno Designer - Empty Layouts", "Old World - Cape Trelawney", "**", "*_empty.ad")

In [26]:
# num_sources = 24
radius = 20
source_size = 4
e = 1e-8

In [27]:
# ensure the scikit methods return the correct townhall coverage
coverage_sklearn = skimage.draw.disk(center=(19.5, 19.5), radius=20)
coverage_actual = np.nonzero(TOWNHALL_COVERAGE)
assert (coverage_sklearn[0] == coverage_actual[0]).all()
assert (coverage_sklearn[1] == coverage_actual[1]).all()

In [28]:
def distance_from_set(s) -> np.ndarray:
    distance = np.where(s, 0.0, np.inf)
    visited = e < s # set grows outward
    for k in itertools.count(1):
        # expand to 3x3 neighbourhood
        visited = e < scipy.ndimage.uniform_filter(np.where(visited, 1.0, 0.0), 3)
        update = (distance == np.inf) & visited
        if not update.any():
            break
        distance[update] = k
    return distance

def signed_distance_from_boundary(s) -> np.ndarray:
    return distance_from_set(s) - np.maximum(0, -1 + distance_from_set(~s))

In [29]:


def vector_to_sources(grid, x, grid_mode):
    
    if grid_mode:

        phi = x[0]
        rotation = np.array([[np.cos(phi), np.sin(phi)], [-np.sin(phi), np.cos(phi)]])
        translation = x[1:3]

        sources = []
        for u in range(-10, 10):
            for v in range(-10, 10):
                coords = np.array((2 * u + (v % 2), np.sqrt(3) * v))
                coords = rotation @ coords + translation
                coords = radius * coords
                coords = coords.astype(int)
                footprint = skimage.draw.rectangle(
                    start=coords - source_size / 2,
                    extent=(source_size, source_size),
                    shape=grid.shape,
                )
                if footprint[0].size and grid[footprint].all():
                    sources.append(coords)
    else: 
        sources = x.reshape((-1, 2))
        sources = sources[0, :] + sources[1:, :]

    return np.array(sources)

    # sources_transformed = list(map(affine_transform, sources))
    # return np.array(sources_transformed)

def sources_to_grid(grid, sources):
    a = np.zeros_like(grid, dtype=int)
    for s in sources:
        d = skimage.draw.disk(center=np.round(s) - .5, radius=radius, shape=a.shape)
        a[d] += 1
    return a

def constraints(distance_grid, sources):
    bounds = np.array(distance_grid.shape) - 1

    for s in sources:

        # ensure coordinates are in bounds
        yield abs(s - np.clip(s, 0, bounds)).sum()

        # ensure building is buildable
        footprint = skimage.draw.rectangle(
            start=np.array(s) - source_size / 2,
            extent=(source_size, source_size),
            shape=distance_grid.shape,
        )
        samples = distance_grid[footprint]
        if samples.size == 0:
            samples = distance_grid
        yield max(0, samples.max(initial=0)) 

    # check lower-triangle-entries of distance matrix
    if sources.size:
        distances = pairwise_distances(np.round(sources))
        distances_tril = distances[np.tril_indices_from(distances, -1)]
        distance_constraints = np.maximum(0, 2 * radius -  distances_tril)
        distance_constraints = np.square(distance_constraints)

        # increase values in (e, 1) to 1 to prevent underconstraining
        distance_constraints = np.where(e < distance_constraints, np.maximum(1, distance_constraints), distance_constraints)

        yield distance_constraints.sum()
    else:
        yield 0

def objective_fn(grid, distance_grid, x, grid_mode, constraint_weight):
    sources = vector_to_sources(grid, x, grid_mode=grid_mode)
    coverage = sources_to_grid(grid, sources)
    reward_coverage = (grid * (coverage > 0)).sum()
    reward_coverage -= (source_size * source_size) * len(sources)
    value = reward_coverage
    value -= constraint_weight * sum(constraints(distance_grid, sources))
    return -value

def sources_to_image(grid, sources):
    output_shape = grid.shape[0], grid.shape[1], 3
    img = np.zeros(output_shape, dtype=np.uint8)
    coverage = sources_to_grid(grid, sources)
    img[:, :, 0] += (255 * (coverage == 0) * (grid == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 0) * (grid == 1)).astype(np.uint8)
    img[:, :, 0] += (255 * (coverage > 1) * (grid == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 1) * (grid == 1)).astype(np.uint8)
    for s in sources:
        r = skimage.draw.rectangle(
            start=np.array(s) - source_size / 2,
            extent=(source_size, source_size),
            shape=grid.shape,
        )
        img[r[0], r[1], :] = 255
    return img

In [30]:
def source_to_ad_object(i, j):
    obj = dict()
    obj["Identifier"] = "Town hall"
    obj["Label"] = ""
    obj["Position"] = f"{i-source_size//2},{j-source_size//2}"
    obj["Size"] = f"{source_size},{source_size}"
    obj["Icon"] = "A7_townhall"
    obj["Template"] = "Guildhouse"
    obj["Color"] = {"A": 255, "R": 238, "G": 130, "B": 238}
    obj["Borderless"] = True
    obj["Road"] = False
    obj["Radius"] = radius
    obj["InfluenceRange"] = -2.0
    obj["BlockedAreaLength"] = 0.0
    obj["BlockedAreaWidth"] = 0.0
    obj["Direction"] = "Up"
    return obj

In [ ]:
def solve(grid, distance_grid):

    best_value = np.inf
    best_sources = None

    for seed in range(3):

        x0 = [0, 0, 0]
        sigma0 = 1.0

        options = cma.CMAOptions()
        options.set("bounds", [[-np.pi, -1, -1], [+np.pi, 1, 1]])
        options.set("seed", seed)
        options.set("popsize", 32)
        options.set("tolflatfitness", 10)

        es = cma.CMAEvolutionStrategy(x0, sigma0, options)
        while not es.stop():
            solutions = es.ask()
            values = [objective_fn(grid, distance_grid, x, grid_mode=True, constraint_weight=0) for x in solutions]
            es.tell(solutions, values)
            es.disp()
            sources = vector_to_sources(grid, es.best.x, grid_mode=True)
            yield sources
        es.result_pretty()

        bounds_lower = 2 * [-1000] + len(sources) * [0, 0]
        bounds_upper = 2 * [+1000] + len(sources) * list(grid.shape)

        x0 = np.concatenate(([0, 0], sources.flatten()))
        sigma0 = 0.1
        constraint_weight = 100

        options = cma.CMAOptions()
        options.set("bounds", [bounds_lower, bounds_upper])
        options.set("seed", seed)
        # options.set("tolstagnation", 10)
        options.set("tolfunhist", 1e-7)
        # options.set("maxiter", 1000)
        options.set("popsize", 64)
        # options.set("tolflatfitness", 32)
        # options.set("tolfun", 0)
        # options.set("tolfunhist", 0)
        options.set("integer_variables", list(range(len(x0))))

        for epoch in range(100):

            es = cma.CMAEvolutionStrategy(x0, sigma0, options)
            # for _ in range(100):
            while not es.stop():
                solutions = es.ask()
                values = [objective_fn(grid, distance_grid, x, grid_mode=False, constraint_weight=constraint_weight) for x in solutions]
                es.tell(solutions, values)
                es.disp()
                sources = vector_to_sources(grid, es.best.x, grid_mode=False)
                yield sources
            es.result_pretty()

            constraint_loss = sum(constraints(distance_grid, sources))
            if 0 == constraint_loss:
                break

            constraint_weight *= 3
            x0 = es.best.x

        if es.best.f < best_value:
            best_value = es.best.f
            best_sources = sources
        else:
            print(f"{seed=} did not yield improvement.")

    yield best_sources
        
    # x_sol = es.best.x
    # x_sol

In [32]:
input_paths = glob.glob(input_glob, recursive=True)
input_paths

['..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\community_island\\community_island_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_01\\moderate_l_01_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_02\\moderate_l_02_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_03\\moderate_l_03_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_04\\moderate_l_04_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_05\\moderate_l_05_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_06\\moderate_l_06_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\L

In [33]:
for input_path in input_paths:

    with open(input_path) as f:
        input_json = json.load(f)
    input_data = ad_to_grid(input_json)

    images = []
    distance_grid = distance_from_set(input_data)

    for iteration, sources in enumerate(solve(input_data, distance_grid)):

        img = sources_to_image(input_data, sources)
        img = np.repeat(img, 2, 0)
        img = np.repeat(img, 2, 1)
        img = np.pad(img, ((100, 0), (0, 0), (0, 0)))
        image = Image.fromarray(img)
        draw = ImageDraw.Draw(image)
        font = ImageFont.truetype("arial.ttf", 16)
        draw.text(
            (0, 0),
            f"Iteration : {iteration}",
            font=font,
        )

        coverage = sources_to_grid(input_data, sources)
        coverage_tiles = ((coverage > 0) & input_data).sum()
        draw.text(
            (0, 20),
            f"Tiles covered : {coverage_tiles}",
            font=font,
        )
        constraint_loss = sum(constraints(distance_grid, sources))
        draw.text(
            (0, 40),
            f"Constraint loss : {round(constraint_loss, 3)}",
            font=font,
        )

        images.append(image)
        image.save(os.path.join("..", "resources", "optimization", "latest.png"))

    output_json = copy.deepcopy(input_json)
    output_json["Objects"] = list(filter(
        lambda obj: obj["Identifier"] in BLOCKING,
        output_json["Objects"]))
    for i, j in sources:
        obj = source_to_ad_object(j, i)
        output_json["Objects"].append(obj)

    output_path = os.path.join("..", "resources", "optimization", "islands", f"{os.path.basename(input_path)}")
    # output_path = input_path.replace("_empty.ad", "_th_opt.ad")
    with open(output_path, "w") as f:
        json.dump(output_json, f, indent=4)

    # animation_path = os.path.join("..", "resources", "optimization", "animation.gif")
    # animation_path = output_path + ".gif"
    # images[0].save(animation_path, save_all=True, append_images=images[1:], duration=1, loop=0)

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=1 (and 1 others) at iteration 0 multiplied by 0.6666555556481476 to stds[1]=0.6666666666666666



(16_w,32)-aCMA-ES (mu_w=9.2,w_1=19%) in dimension 3 (seed=620736, Sat Oct  4 07:15:28 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     32 -2.286900000000000e+04 1.0e+00 8.15e-01  5e-01  8e-01 0:00.5
    2     64 -2.253500000000000e+04 1.5e+00 6.99e-01  3e-01  6e-01 0:01.0
    3     96 -2.184700000000000e+04 1.8e+00 7.47e-01  4e-01  7e-01 0:01.4
   10    320 -2.257700000000000e+04 3.8e+00 9.25e-01  4e-01  1e+00 0:04.7
   19    608 -2.296500000000000e+04 6.9e+00 1.45e+00  7e-01  2e+00 0:08.9
   30    960 -2.196000000000000e+04 1.0e+01 1.57e+00  5e-01  2e+00 0:14.0
   43   1376 -2.237900000000000e+04 1.5e+01 1.44e+00  2e-01  7e-01 0:20.0
   59   1888 -2.172900000000000e+04 5.5e+01 3.06e+00  7e-01  2e+00 0:27.4
   76   2432 -2.287800000000000e+04 1.1e+02 4.99e+00  7e-01  2e+00 0:35.4
   96   3072 -2.242200000000000e+04 7.6e+01 7.53e+00  2e-01  5e-01 0:44.8
  100   3200 -2.225100000000000e+04 6.2e+01 7.18e+00  1e-01  4e-01 0:46.6
  124   3968 -2.246800

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=0 (and 45 others) at iteration 0 multiplied by 2.0 to stds[0]=0.2



    2    128 -2.115100000000000e+04 1.1e+00 9.84e-02  2e-01  2e-01 0:00.4
    3    192 -2.128627892956873e+04 1.2e+00 1.05e-01  2e-01  2e-01 0:00.5
   21   1344 -2.142539774096348e+04 1.7e+00 3.45e-01  7e-01  9e-01 0:03.7
   45   2880 -2.181587892683474e+04 2.3e+00 4.19e-01  8e-01  1e+00 0:07.8
   75   4800 -2.224000000000000e+04 2.5e+00 2.67e-01  5e-01  7e-01 0:12.9
  100   6400 -2.246700000000000e+04 2.6e+00 1.97e-01  4e-01  5e-01 0:17.2
  141   9024 -2.288500000000000e+04 2.8e+00 1.17e-01  2e-01  4e-01 0:24.3
  188  12032 -2.298200000000000e+04 3.4e+00 1.04e-01  2e-01  4e-01 0:32.4
  200  12800 -2.298900000000000e+04 3.5e+00 9.56e-02  2e-01  4e-01 0:34.5
  258  16512 -2.303300000000000e+04 4.6e+00 2.71e-02  2e-01  2e-01 0:44.4
termination on tolfunhist=1e-07
final/bestever f-value = -2.303300e+04 -2.303300e+04 after 16512/14282 evaluations
incumbent solution: [  2.  -1.  54. 234. 101. 229.  46. 170. ...]
std deviations: [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 ...]
(16_w,32)-aCMA-ES (mu_w=9

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=1 (and 1 others) at iteration 0 multiplied by 0.6666555556481476 to stds[1]=0.6666666666666666



...


   53   3392 -3.096000000000000e+03 1.2e+01 2.31e-01  5e-01  5e-01 0:02.8
termination on tolfunhist=1e-07
final/bestever f-value = -3.096000e+03 -3.096000e+03 after 3392/2015 evaluations
incumbent solution: [-0. -1. 39.  8. 63. 39. 23. 45. ...]
std deviations: [0.51694238 0.53420601 0.49932148 0.5239144  0.46305941 0.52464965
 0.45826148 0.52441716 ...]
(16_w,32)-aCMA-ES (mu_w=9.2,w_1=19%) in dimension 3 (seed=2, Sat Oct  4 12:11:28 2025)


c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=1 (and 1 others) at iteration 0 multiplied by 0.6666555556481476 to stds[1]=0.6666666666666666



Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     32 -2.991000000000000e+03 1.0e+00 9.53e-01  6e-01  8e-01 0:00.4
    2     64 -2.752000000000000e+03 1.5e+00 8.39e-01  6e-01  8e-01 0:00.8
    3     96 -3.110000000000000e+03 1.5e+00 8.91e-01  5e-01  9e-01 0:01.2
   11    352 -3.086000000000000e+03 3.5e+00 8.23e-01  3e-01  7e-01 0:04.6
   21    672 -3.016000000000000e+03 2.9e+00 7.30e-01  1e-01  3e-01 0:08.7
   34   1088 -3.125000000000000e+03 6.9e+00 1.07e+00  6e-02  1e-01 0:14.1
   49   1568 -3.136000000000000e+03 1.2e+01 1.01e+00  1e-02  4e-02 0:20.4
   66   2112 -3.135000000000000e+03 1.2e+01 1.24e+00  3e-03  8e-03 0:27.6
   69   2208 -3.135000000000000e+03 1.6e+01 1.01e+00  3e-03  8e-03 0:28.8
termination on tolfunhist=1e-12
final/bestever f-value = -3.135000e+03 -3.140000e+03 after 2208/1112 evaluations
incumbent solution: [-0.88689412, -0.04880188, 0.78638398]
std deviation: [0.00316935, 0.00479061, 0.00792906]
(32_w,64)-aCMA-ES (mu_w=17.6,w_1=11%) 

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=0 (and 9 others) at iteration 0 multiplied by 2.0 to stds[0]=0.2



   51   3264 -3.081000000000000e+03 5.3e+00 1.44e-01  2e-01  2e-01 0:02.7
termination on tolfunhist=1e-07
final/bestever f-value = -3.081000e+03 -3.081000e+03 after 3264/1617 evaluations
incumbent solution: [-0. -1. 38.  8. 62. 40. 22. 45. ...]
std deviations: [0.24592135 0.22989903 0.20835038 0.21628455 0.21577114 0.20332924
 0.21409683 0.21740854 ...]
epoch=0 did not yield improvement.
